# ABS Quarterly Financial Accounts 5232

## Python set-up

In [1]:
# system imports
import textwrap

# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol

# local imports
from abs_helper import get_abs_data
from plotting import line_plot

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("5232.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

In [3]:
# list tables
textwrap.wrap(", ".join(abs_dict.keys()), width=80)

['5232001, 5232002, 5232003, 5232004, 5232005, 5232006, 5232007, 5232008, 5232009,',
 '5232010, 5232011, 5232012, 5232013, 5232014, 5232015, 5232016, 5232017, 5232018,',
 '5232019, 5232020, 5232021, 5232022, 5232023, 5232024, 5232025, 5232026, 5232027,',
 '5232028, 5232029, 5232030, 5232031, 5232032, 5232033, 5232034, 5232035, 5232036,',
 '5232037, 5232038, 5232039, 5232040, 5232041, 5232042, 5232043, 5232044, 5232045,',
 '5232046, 5232047, 5232048, 5232049, 5232050, 5232051, 5232052, 5232053, 5232054']

## Plot

### Household balance sheet

In [4]:
def plot_hbs() -> None:
    """Plot the household balance sheet data."""

    table = "5232035"
    data = abs_dict[table]
    series_type = "Original"
    plots = meta[meta["Table"] == table][metacol.did]

    for plot in plots:
        search = {series_type: metacol.stype, plot: metacol.did, table: metacol.table}
        _table, ident, orig_units = ra.find_id(meta, search)

        # manage long wordy chart titles
        max_length = 60
        title = f"HBS: {plot[:-2]}"
        footnote = ""
        replaceable = {
            "- Produced - Fixed assets -": ["- PFA -", " PFA = Produced fixed assets."],
            "- Non-produced assets -": ["- NPA -", " NPA = Non-produced assets."],
            "- Non-financial -": ["- NF -", " NF = Non-financial."],
            "- Fixed assets -": ["- FA -", " FA = Fixed assets."],
            "Financial assets -": ["FA -", " FA = Financial assets."],
        }
        for foundling, [replacement, fnote] in replaceable.items():
            if foundling in title and len(title) >= max_length:
                title = title.replace(foundling, replacement)
                footnote = f"{footnote}{fnote}"
        title = textwrap.fill(title, width=max_length)

        series = data[ident]
        series, units = ra.recalibrate(series, orig_units)

        # plot
        line_plot(
            series,
            starts=plot_times,
            tags=plot_tags,
            title=title,
            ylabel=f"{units} current prices",
            rfooter=source,
            lfooter=f"Australia. {series_type.capitalize()} series."
            f" HBS = Household balance sheet.{footnote}",
            show=SHOW,
        )


plot_hbs()

In [5]:
def plot_lad() -> None:
    """Plot the household balance sheet data for land and dwellings."""

    # Land and dwelling assets - side by side
    table = "5232035"
    data = abs_dict[table]
    series_type = "Original"
    land = "Non-financial - Non-produced assets - Land ;"
    dwellings = "Non-financial - Produced - Fixed assets - Dwellings ;"
    combined = {}
    for name, series in zip(("Land", "Dwellings"), (land, dwellings)):
        search = {series_type: metacol.stype, series: metacol.did, table: metacol.table}
        _table, ident, units = ra.find_id(meta, search)
        combined[name] = data[ident]
    cdf = pd.DataFrame(combined)
    cdf, units = ra.recalibrate(cdf, units)
    title = "Household Balance Sheet for Land and Dwellings"
    line_plot(
        cdf,
        title=title,
        ylabel=f"{units} current prices",
        rfooter=source,
        lfooter=f"Australia. {series_type.capitalize()} series",
        show=SHOW,
    )


plot_lad()

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sat Jul 06 2024 11:57:06

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

pandas : 2.2.2
readabs: 0.0.6a3

Watermark: 2.4.3



In [7]:
print("Finished")

Finished
